In [1]:
import osmnx as ox
import pandas as pd
import time
import requests
import os
from pathlib import Path
from datetime import datetime

 # 1. Get list of all US Counties 
 
The attempt to download data for a very large county - Los Angeles - was successful.
 
Now, i will attempt to obtain a list of all US Counties...

I should be able to get this data from the US Census API Geography Endpoint: https://www.census.gov/data/developers/data-sets/geo-info.html

In [2]:
census_api_key = " 349179f9505abb551eb4de4ab998d2a0f58d06aa"

In [3]:
url = 'https://api.census.gov/data/2023/geoinfo'

# needed! 
headers = {'User-Agent': 'Mozilla/5.0'}

params = {'get':'NAME', 
                        'for': 'county:*',
                      }

response = requests.get(url, headers=headers, params=params, timeout=10)

if response.status_code == 200:
    data = response.json()
    print("Success")
else:
    print(response.status_code)

Success


Convert to a Pandas dataframe

In [4]:
# extract county names
counties_list = [item[0] for item in data][1:]

counties_df = pd.DataFrame(counties_list, columns=['name_full'])

# add State column 
counties_df['state'] = counties_df.apply(lambda x: x['name_full'].split(",")[1].strip(), axis=1)
# add Counties
counties_df['county'] = counties_df.apply(lambda x: x['name_full'].split(",")[0].strip(), axis=1)

counties_df.head()

,name_full,state,county
0,"Autauga County, Alabama",Alabama,Autauga County
1,"Baldwin County, Alabama",Alabama,Baldwin County
2,"Barbour County, Alabama",Alabama,Barbour County
3,"Bibb County, Alabama",Alabama,Bibb County
4,"Blount County, Alabama",Alabama,Blount County


### Save the County names to file

In [5]:
counties_df.to_csv('us_counties.csv', index = False)

# 2. Download all US Counies

In [6]:
counties_df = pd.read_csv('us_counties.csv')

# df for results
results_df = pd.DataFrame(columns = ['state', 'name_full', 'name_abbrev', 'downloaded', 'file_path', 'size', 'date', 'area'])


In [7]:
counties_df.head()

,name_full,state,county
0,"Autauga County, Alabama",Alabama,Autauga County
1,"Baldwin County, Alabama",Alabama,Baldwin County
2,"Barbour County, Alabama",Alabama,Barbour County
3,"Bibb County, Alabama",Alabama,Bibb County
4,"Blount County, Alabama",Alabama,Blount County


In [8]:
def get_abbrev_name(county_row):
    county = county_row['name_full'].split(",")[0]

    # remove 'County' suffix
    suffix_loc = county.find("County")

    if suffix_loc != -1:
        county = county[:suffix_loc].strip()
    
    return county

In [ ]:
import requests
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import shape
import pandas as pd
from datetime import datetime

# ---- configure ----
base_url = "https://server7.tplgis.org/arcgis7/rest/services/ParkServe/ParkServe_ProdNew/MapServer/2/query" # replace with your URL
base_condition = "(park_designation = 'LP' OR park_designation = 'LREC')"

# fields, format, page size
out_fields = "*"   # or list of fields you need
page_size = 1000   # how many features to request per page (tune for the service)
equal_area_crs = "EPSG:5070"   # Conterminous US Albers (good for US-wide comparisons)

# list of counties to query (string names). You can take these from your counties_df
counties = counties_df['name_full'].unique().tolist()

def fetch_features_for_county(full_name):
    """Fetch all features for a county (pages) and return a list of GeoJSON geometries (dict)"""
    features = []
    offset = 0
    county, state = [s.strip() for s in full_name.split(",")]
    while True:
        where_clause = f"Park_County = '{county.strip()}' AND park_state = '{state.strip()}' AND {base_condition}"
        params = {
            "where": where_clause,
            "outFields": out_fields,
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": page_size,
        }
        r = requests.get(base_url, params=params, timeout=60)
        r.raise_for_status()
        gj = r.json()

        # geojson feature collection -> features list
        page_features = gj.get("features", [])
        if not page_features:
            break

        features.extend(page_features)

        # If the server returns less than page_size we are done;
        # Some ArcGIS services set "exceededTransferLimit": true — in many cases this
        # means you must page using resultOffset until no more features returned.
        if len(page_features) < page_size:
            break
        offset += page_size

    return features

def area_for_features_geojson(features_geojson, equal_area_crs=equal_area_crs):
    """Given a list of GeoJSON features (dicts), compute union area in m^2 efficiently."""
    if not features_geojson:
        return 0.0

    # Build a GeoSeries of geometries (avoid heavy attribute copying)
    geoms = [shape(feat["geometry"]) for feat in features_geojson if feat.get("geometry")]

    if not geoms:
        return 0.0
    
    gsf = gpd.GeoSeries(geoms)
    
    # ensure CRS is set before transforming
    if gsf.crs is None:
        gsf = gsf.set_crs("EPSG:4326")

    # Some features may be invalid; attempt a quick fix (buffer(0)) lazily
    # quick geometry fix if needed
    if not gsf.is_valid.all():
        gsf = gsf.buffer(0)
    # Reproject to equal-area CRS once and unary_union
    gsf_proj = gsf.to_crs(equal_area_crs)
    union_geom = unary_union(gsf_proj.values)   # single unioned geometry
    area_m2 = float(union_geom.area) if union_geom is not None else 0.0
    return area_m2

# ---- main loop: do one county at a time to limit memory ----
rows = []
for county in counties:
    try:
        features = fetch_features_for_county(county)
        area_m2 = area_for_features_geojson(features)
        rows.append({"county": county, "area_m2": area_m2})
    except Exception as e:
        print(f"ERROR fetching {county}: {e}")
        rows.append({"county": county, "area_m2": float('nan')})

# final tidy DataFrame
area_df = pd.DataFrame(rows)
area_df = area_df.sort_values("area_m2", ascending=False).reset_index(drop=True)
print(area_df.head())

# optional: merge with your counties_df for additional fields
# merged = counties_df.merge(area_df, left_on="name_full", right_on="county", how="left")


In [12]:
area_df.head(20)

,county,area_m2
0,"Collier County, Florida",2.361040e+09
1,"Maricopa County, Arizona",4.099900e+08
2,"Los Angeles County, California",3.201923e+08
3,"Santa Clara County, California",3.031208e+08
4,"San Diego County, California",2.616306e+08
5,"Harris County, Texas",2.178562e+08
6,"Pima County, Arizona",2.054264e+08
7,"King County, Washington",1.902062e+08
8,"Davidson County, Tennessee",1.783267e+08
9,"Dallas County, Texas",1.526676e+08


In [13]:
print(f"total area: {area_df['area_m2'].sum()}")

total area: 15991018528.014133


In [23]:
import requests
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd
from datetime import datetime

# --- Config ---
equal_area_crs = "EPSG:5070"  # Equal-area CRS for area calculations
today = datetime.today().strftime("%Y-%m-%d")

# Census TIGER base URLs (official)
COUNTY_URL = "https://www2.census.gov/geo/tiger/TIGER2025/COUNTY/tl_2025_us_county.zip"
PLACE_BASE_URL = "https://www2.census.gov/geo/tiger/TIGER2025/PLACE/tl_2025_{statefp}_place.zip"

# ParkServe endpoint
PARKSERVE_URL = "https://server7.tplgis.org/arcgis7/rest/services/ParkServe/ParkServe_ProdNew/MapServer/2/query"
PARKSERVE_CONDITION = "(park_designation = 'LP' OR park_designation = 'LREC')"

# --- Load all counties (nationwide shapefile) ---
print("Loading counties...")
counties_gdf = gpd.read_file(COUNTY_URL).to_crs("EPSG:4326")

# Create readable columns
counties_gdf["statefp"] = counties_gdf["STATEFP"]
counties_gdf["countyfp"] = counties_gdf["COUNTYFP"]
counties_gdf["geoid"] = counties_gdf["GEOID"]
counties_gdf["county_name"] = counties_gdf["NAME"]
counties_gdf["name_full"] = (
    counties_gdf["county_name"] + ", " + counties_gdf["STATEFP"].map(str)
)
counties_gdf = counties_gdf.to_crs(equal_area_crs)

# --- Load all cities ("places") ---
print("Loading city shapefiles by state...")
state_fips_list = counties_gdf["statefp"].unique().tolist()

city_gdfs = []
for fips in state_fips_list:
    url = PLACE_BASE_URL.format(statefp=fips.zfill(2))
    try:
        gdf = gpd.read_file(url)
        gdf["statefp"] = fips
        gdf = gdf.to_crs(equal_area_crs)
        city_gdfs.append(gdf)
    except Exception as e:
        print(f"  ⚠️ Failed to load {url}: {e}")

cities_gdf = pd.concat(city_gdfs, ignore_index=True)
print(f"Loaded {len(cities_gdf)} city polygons across {len(state_fips_list)} states.")

# --- ParkServe helper function ---
def fetch_features_for_county(county_name, state_fips):
    """Fetch ParkServe parks by county and state FIPS"""
    features = []
    offset = 0
    while True:
        where_clause = (
            f"Park_State_FIPS = '{state_fips}' AND Park_County = '{county_name} County' AND {PARKSERVE_CONDITION}"
        )
        params = {
            "where": where_clause,
            "outFields": "*",
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": 1000,
        }
        r = requests.get(PARKSERVE_URL, params=params, timeout=60)
        r.raise_for_status()
        gj = r.json()
        page_features = gj.get("features", [])
        if not page_features:
            break
        features.extend(page_features)
        if len(page_features) < 1000:
            break
        offset += 1000
    return features


def parks_within_cities(features_geojson, city_gdf):
    """Intersect ParkServe features with city boundaries and sum area within them."""
    if not features_geojson:
        return 0.0

    parks = gpd.GeoDataFrame(
        geometry=[shape(f["geometry"]) for f in features_geojson if f.get("geometry")],
        crs="EPSG:4326",
    ).to_crs(equal_area_crs)

    # Fix invalid geometries
    parks["geometry"] = parks.buffer(0)
    parks = parks[~parks.geometry.is_empty]

    # Intersect with cities (same CRS)
    parks_in_cities = gpd.overlay(parks, city_gdf, how="intersection")

    if parks_in_cities.empty:
        return 0.0

    # Compute total area (in m²)
    return float(parks_in_cities.geometry.area.sum())


# --- Main loop ---
rows = []
for _, row in counties_gdf.iterrows():
    county_name = row["county_name"]
    state_fips = row["statefp"]
    geoid = row["geoid"]

    print(f"Processing {county_name} (state FIPS {state_fips})...")

    try:
        features = fetch_features_for_county(county_name, state_fips)

        # Get all cities within this county geometry
        county_geom = row["geometry"]
        cities_in_county = cities_gdf[cities_gdf.intersects(county_geom)]

        area_m2 = parks_within_cities(features, cities_in_county)
        rows.append(
            {
                "county": county_name,
                "state_fips": state_fips,
                "county_fips": row["countyfp"],
                "geoid": geoid,
                "area_m2_citybounded": area_m2,
                "date": today,
            }
        )

        print(f"  → area within cities = {area_m2:,.1f} m²")

    except Exception as e:
        print(f"  ❌ ERROR processing {county_name}: {e}")
        rows.append(
            {
                "county": county_name,
                "state_fips": state_fips,
                "county_fips": row["countyfp"],
                "geoid": geoid,
                "area_m2_citybounded": float("nan"),
                "date": today,
            }
        )

# --- Results ---
bounded_area_df = pd.DataFrame(rows).sort_values("area_m2_citybounded", ascending=False)
print("\nTop 5 counties by park area within cities:")
print(bounded_area_df.head())


Loading counties...
Loading city shapefiles by state...
Loaded 32629 city polygons across 56 states.
Processing Kiowa (state FIPS 40)...
  → area within cities = 208,671.6 m²
Processing Lake (state FIPS 46)...
  → area within cities = 295,319.2 m²
Processing Caswell (state FIPS 37)...
  → area within cities = 59,291.8 m²
Processing Presidio (state FIPS 48)...
  → area within cities = 70,501.7 m²
Processing Greene (state FIPS 39)...
  → area within cities = 6,266,808.3 m²
Processing Willacy (state FIPS 48)...
  → area within cities = 126,703.1 m²
Processing Bayamón (state FIPS 72)...
  → area within cities = 0.0 m²
Processing Los Alamos (state FIPS 35)...
  → area within cities = 1,202,611.8 m²
Processing Choctaw (state FIPS 40)...
  → area within cities = 79,772.6 m²
Processing Dickinson (state FIPS 20)...
  → area within cities = 376,041.8 m²
Processing Starr (state FIPS 48)...
  → area within cities = 262,273.8 m²
Processing Effingham (state FIPS 13)...
  → area within cities = 318,3

In [26]:
print(f"total area: {bounded_area_df['area_m2_citybounded'].sum()}")

total area: 9208931150.99102


In [28]:
import requests
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd
from datetime import datetime

# --- Config ---
equal_area_crs = "EPSG:5070"  # Equal-area CRS for area calculations
today = datetime.today().strftime("%Y-%m-%d")

# Census TIGER base URLs (official)
COUNTY_URL = "https://www2.census.gov/geo/tiger/TIGER2025/COUNTY/tl_2025_us_county.zip"
PLACE_BASE_URL = "https://www2.census.gov/geo/tiger/TIGER2025/PLACE/tl_2025_{statefp}_place.zip"

# ParkServe endpoint
PARKSERVE_URL = "https://server7.tplgis.org/arcgis7/rest/services/ParkServe/ParkServe_ProdNew/MapServer/2/query"
PARKSERVE_CONDITION = "(park_designation = 'LP' OR park_designation = 'LREC')"

# --- Load all counties (nationwide shapefile) ---
print("Loading counties...")
counties_gdf = gpd.read_file(COUNTY_URL).to_crs("EPSG:4326")

# Create readable columns
counties_gdf["statefp"] = counties_gdf["STATEFP"]
counties_gdf["countyfp"] = counties_gdf["COUNTYFP"]
counties_gdf["geoid"] = counties_gdf["GEOID"]
counties_gdf["county_name"] = counties_gdf["NAME"]
counties_gdf["name_full"] = (
    counties_gdf["county_name"] + ", " + counties_gdf["STATEFP"].map(str)
)
counties_gdf = counties_gdf.to_crs(equal_area_crs)

# --- Load all cities ("places") ---
print("Loading city shapefiles by state...")
state_fips_list = counties_gdf["statefp"].unique().tolist()

city_gdfs = []
for fips in state_fips_list:
    url = PLACE_BASE_URL.format(statefp=fips.zfill(2))
    try:
        gdf = gpd.read_file(url)
        gdf["statefp"] = fips
        gdf = gdf.to_crs(equal_area_crs)
        city_gdfs.append(gdf)
    except Exception as e:
        print(f"  ⚠️ Failed to load {url}: {e}")

cities_gdf = pd.concat(city_gdfs, ignore_index=True)
print(f"Loaded {len(cities_gdf)} city polygons across {len(state_fips_list)} states.")

# --- ParkServe helper function ---
def fetch_features_for_county(county_name, state_fips):
    """Fetch ParkServe parks by county and state FIPS"""
    features = []
    offset = 0
    while True:
        where_clause = (
            f"Park_State_FIPS = '{state_fips}' AND Park_County = '{county_name} County'"
        )
        params = {
            "where": where_clause,
            "outFields": "*",
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": 1000,
        }
        r = requests.get(PARKSERVE_URL, params=params, timeout=60)
        r.raise_for_status()
        gj = r.json()
        page_features = gj.get("features", [])
        if not page_features:
            break
        features.extend(page_features)
        if len(page_features) < 1000:
            break
        offset += 1000
    return features


def parks_within_cities(features_geojson, city_gdf):
    """Intersect ParkServe features with city boundaries and sum area within them."""
    if not features_geojson:
        return 0.0

    parks = gpd.GeoDataFrame(
        geometry=[shape(f["geometry"]) for f in features_geojson if f.get("geometry")],
        crs="EPSG:4326",
    ).to_crs(equal_area_crs)

    # Fix invalid geometries
    parks["geometry"] = parks.buffer(0)
    parks = parks[~parks.geometry.is_empty]

    # Intersect with cities (same CRS)
    parks_in_cities = gpd.overlay(parks, city_gdf, how="intersection")

    if parks_in_cities.empty:
        return 0.0

    # Compute total area (in m²)
    return float(parks_in_cities.geometry.area.sum())


# --- Main loop ---
rows_all = []
for _, row in counties_gdf.iterrows():
    county_name = row["county_name"]
    state_fips = row["statefp"]
    geoid = row["geoid"]

    try:
        features = fetch_features_for_county(county_name, state_fips)

        # Get all cities within this county geometry
        county_geom = row["geometry"]
        cities_in_county = cities_gdf[cities_gdf.intersects(county_geom)]

        area_m2 = parks_within_cities(features, cities_in_county)
        rows_all.append(
            {
                "county": county_name,
                "state_fips": state_fips,
                "county_fips": row["countyfp"],
                "geoid": geoid,
                "area_m2_citybounded": area_m2,
                "date": today,
            }
        )

    except Exception as e:
        print(f"  ❌ ERROR processing {county_name}: {e}")
        rows_all.append(
            {
                "county": county_name,
                "state_fips": state_fips,
                "county_fips": row["countyfp"],
                "geoid": geoid,
                "area_m2_citybounded": float("nan"),
                "date": today,
            }
        )

# --- Results ---
bounded_area_df_all = pd.DataFrame(rows).sort_values("area_m2_citybounded", ascending=False)
print("\nTop 5 counties by park area within cities:")
print(bounded_area_df.head())


Loading counties...
Loading city shapefiles by state...
Loaded 32629 city polygons across 56 states.

Top 5 counties by park area within cities:
           county state_fips county_fips  geoid  area_m2_citybounded  \
1918     Maricopa         04         013  04013         3.087466e+08   
1993  Los Angeles         06         037  06037         2.504030e+08   
1896    San Diego         06         073  06073         1.851398e+08   
1574       Harris         48         201  48201         1.832321e+08   
2298       Dallas         48         113  48113         1.534253e+08   

            date  
1918  2025-10-29  
1993  2025-10-29  
1896  2025-10-29  
1574  2025-10-29  
2298  2025-10-29  


In [29]:
print(f"total area: {bounded_area_df_all['area_m2_citybounded'].sum()}")

total area: 9208931150.99102


In [53]:
import requests
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd
from datetime import datetime

# --- Config ---
equal_area_crs = "EPSG:5070"  # Equal-area CRS for accurate area
today = datetime.today().strftime("%Y-%m-%d")

# --- TIGER/Line Sources ---
COUNTY_URL = "https://www2.census.gov/geo/tiger/TIGER2025/COUNTY/tl_2025_us_county.zip"
PLACE_BASE_URL = "https://www2.census.gov/geo/tiger/TIGER2025/PLACE/tl_2025_{statefp}_place.zip"

# --- ParkServe Endpoint ---
PARKSERVE_URL = "https://server7.tplgis.org/arcgis7/rest/services/ParkServe/ParkServe_ProdNew/MapServer/2/query"
PARKSERVE_CONDITION = "(park_designation = 'LP' OR park_designation = 'LREC')"

# --- Load all counties ---
print("Loading counties...")
counties_gdf = gpd.read_file(COUNTY_URL).to_crs("EPSG:4326")
counties_gdf["statefp"] = counties_gdf["STATEFP"]
counties_gdf["countyfp"] = counties_gdf["COUNTYFP"]
counties_gdf["geoid"] = counties_gdf["GEOID"]
counties_gdf["county_name"] = counties_gdf["NAME"]
counties_gdf = counties_gdf.to_crs(equal_area_crs)

# --- Load all cities (places) ---
print("Loading city shapefiles by state...")
state_fips_list = counties_gdf["statefp"].unique().tolist()  # ✅ Only valid states
city_gdfs = []

for fips in state_fips_list:
    url = PLACE_BASE_URL.format(statefp=fips.zfill(2))
    try:
        gdf = gpd.read_file(url)
        gdf["statefp"] = fips
        gdf = gdf.to_crs(equal_area_crs)
        city_gdfs.append(gdf)
    except Exception as e:
        print(f"  ⚠️ Failed to load {url}: {e}")

cities_gdf = pd.concat(city_gdfs, ignore_index=True)
print(f"✅ Loaded {len(cities_gdf)} city polygons across {len(state_fips_list)} states.")

# --- ParkServe Fetch Helper ---
def fetch_features_for_state(state_fips):
    """Fetch ParkServe features by state FIPS (entire state)."""
    features = []
    offset = 0
    while True:
        params = {
            "where": f"Park_State_FIPS = '{state_fips}' AND {PARKSERVE_CONDITION}",
            "outFields": "*",
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": 1000,
        }
        r = requests.get(PARKSERVE_URL, params=params, timeout=60)
        r.raise_for_status()
        gj = r.json()
        page = gj.get("features", [])
        if not page:
            break
        features.extend(page)
        if len(page) < 1000:
            break
        offset += 1000
    return features

def park_area_within_city(parks_gdf, city_geom):
    """Compute total park area within a single city geometry."""
    parks_in_city = gpd.overlay(parks_gdf, gpd.GeoDataFrame(geometry=[city_geom], crs=equal_area_crs), how="intersection")
    return float(parks_in_city.geometry.area.sum()) if not parks_in_city.empty else 0.0

# --- Main Loop ---
rows_all = []

for state_fips in state_fips_list:
    print(f"Processing state {state_fips}...")
    try:
        features = fetch_features_for_state(state_fips)
        if not features:
            print(f"  ⚠️ No parks found for state {state_fips}.")
            continue

        # Convert parks to GeoDataFrame (equal-area CRS)
        parks = gpd.GeoDataFrame(
            geometry=[shape(f["geometry"]) for f in features if f.get("geometry")],
            crs="EPSG:4326",
        ).to_crs(equal_area_crs)

        # Fix invalid geometries
        parks["geometry"] = parks.buffer(0)
        parks = parks[~parks.geometry.is_empty]

        # Cities for this state
        cities_state = cities_gdf[cities_gdf["statefp"] == state_fips]

        for _, city in cities_state.iterrows():
            area_m2 = park_area_within_city(parks, city.geometry)
            rows_all.append(
                {
                    "state_fips": state_fips,
                    "placefp": city["PLACEFP"],
                    "city_name": city["NAME"],
                    "area_m2_parks": area_m2,
                    "date": today,
                }
            )

    except Exception as e:
        print(f"  ❌ ERROR processing state {state_fips}: {e}")

# --- Results ---
bounded_area_df = pd.DataFrame(rows_all).sort_values("area_m2_parks", ascending=False)
print(f"\n✅ Completed. {len(bounded_area_df)} cities processed.")
print("\nTop 5 cities by total park area:")
print(bounded_area_df.head())

# (optional) save to file
bounded_area_df.to_csv(f"parkserve_city_areas_{today}.csv", index=False)


Loading counties...
Loading city shapefiles by state...
✅ Loaded 32629 city polygons across 56 states.
Processing state 40...
Processing state 46...
Processing state 37...
Processing state 48...
Processing state 39...
Processing state 72...
Processing state 35...
Processing state 20...
Processing state 13...
Processing state 55...
Processing state 01...
Processing state 47...
Processing state 51...
Processing state 05...
Processing state 22...
Processing state 12...
Processing state 21...
Processing state 32...
Processing state 06...
Processing state 28...
Processing state 31...
Processing state 27...
Processing state 19...
Processing state 33...
Processing state 08...
Processing state 36...
Processing state 53...
Processing state 41...
Processing state 29...
Processing state 18...
Processing state 38...
Processing state 30...
Processing state 26...
Processing state 44...
Processing state 17...
Processing state 04...
Processing state 02...
Processing state 54...
Processing state 45...


In [54]:
bounded_area_df.area_m2_parks.sum()

9655146991.715569

In [55]:
bounded_area_df.head()

,state_fips,placefp,city_name,area_m2_parks,date
25633,04,55000,Phoenix,1.670919e+08,2025-10-30
2555,48,35000,Houston,1.662058e+08,2025-10-30
9140,47,52006,Nashville-Davidson metropolitan government (ba...,1.142194e+08,2025-10-30
13158,06,66000,San Diego,1.062226e+08,2025-10-30
12942,06,44000,Los Angeles,9.020624e+07,2025-10-30


In [84]:
pop_file = pd.read_csv("data/sub-est2024.csv", sep=",", encoding="latin1")


In [93]:
import pandas as pd

# Load both datasets
parks_df = bounded_area_df
pop_df = pd.read_csv("data/sub-est2024.csv", encoding="latin1")

# Keep only "place" rows (SUMLEV 162 = incorporated places, 157 = county subdivisions)
pop_places = pop_df[pop_df["SUMLEV"].isin([157, 162])].copy()

# Zero-pad state and place codes
pop_places["STATEFP"] = pop_places["STATE"].astype(str).str.zfill(2)
pop_places["PLACEFP"] = pop_places["PLACE"].astype(str).str.zfill(5)

parks_df["statefp"] = parks_df["state_fips"].astype(str).str.zfill(2)
parks_df["placefp"] = parks_df["placefp"].astype(str).str.zfill(5)

# Create a join key
pop_places["place_id"] = pop_places["STATEFP"] + pop_places["PLACEFP"]
parks_df["place_id"] = parks_df["statefp"] + parks_df["placefp"]

# Merge population into parks data
merged = parks_df.merge(
    pop_places[["place_id", "NAME", "POPESTIMATE2024"]],
    on="place_id",
    how="left"
)

# Rename for clarity
merged = merged.rename(columns={
    "NAME": "place_name",
    "POPESTIMATE2024": "population_2024"
})

print(merged.head())


  state_fips placefp city_name  area_m2_parks        date statefp place_id  \
0         04   55000   Phoenix   1.670919e+08  2025-10-30      04  0455000   
1         04   55000   Phoenix   1.670919e+08  2025-10-30      04  0455000   
2         48   35000   Houston   1.662058e+08  2025-10-30      48  4835000   
3         48   35000   Houston   1.662058e+08  2025-10-30      48  4835000   
4         48   35000   Houston   1.662058e+08  2025-10-30      48  4835000   

           place_name  population_2024  
0        Phoenix city        1673164.0  
1        Phoenix city        1673164.0  
2        Houston city        2390125.0  
3  Houston city (pt.)          43887.0  
4  Houston city (pt.)        2339948.0  


In [98]:
high_population_places = merged[merged.population_2024 > 250000]
medium_population_places = merged[(merged["population_2024"] > 100000) & (merged["population_2024"] < 250000)]
low_population_places = merged[merged.population_2024 < 100000]

In [112]:
high_population_places.describe()

,area_m2_parks,population_2024
count,1.900000e+02,1.900000e+02
mean,2.933799e+07,6.705999e+05
std,3.150416e+07,8.144857e+05
min,1.493510e+06,2.502660e+05
25%,1.088941e+07,3.039632e+05
50%,1.903634e+07,4.230235e+05
75%,3.436704e+07,6.810228e+05
max,1.670919e+08,8.478072e+06


In [107]:
medium_population_places.describe()

,area_m2_parks,population_2024
count,4.920000e+02,492.000000
mean,5.762534e+06,144295.067073
std,7.126514e+06,38967.789121
min,9.894575e+04,100128.000000
25%,1.953298e+06,112422.000000
50%,3.735783e+06,134342.000000
75%,6.923462e+06,168477.750000
max,6.440828e+07,246170.000000


In [108]:
low_population_places.describe()

,area_m2_parks,population_2024
count,3.951800e+04,39518.000000
mean,3.018703e+05,5827.820942
std,2.305015e+06,12977.704873
min,0.000000e+00,0.000000
25%,0.000000e+00,315.000000
50%,0.000000e+00,1037.000000
75%,1.271156e+05,4432.000000
max,1.662058e+08,99999.000000


In [2]:
import pandas as pd
city_parks = pd.read_csv('output/city_parks_contiguous_2025-11-02.csv')

In [3]:
city_parks

,statefp,placefp,city_name,osm_area_m2,parkserve_area_m2,combined_area_m2,overlap_area_m2,date
0,1,17176,Coosada,0.0,0.0,0.0,0.0,2025-11-02
1,1,9736,Brookside,0.0,0.0,0.0,0.0,2025-11-02
2,1,64152,Rehobeth,0.0,0.0,0.0,0.0,2025-11-02
3,1,81720,Wetumpka,0.0,0.0,0.0,0.0,2025-11-02
4,1,460,Adamsville,0.0,0.0,0.0,0.0,2025-11-02
...,...,...,...,...,...,...,...,...
31535,56,55200,Mountain View,0.0,0.0,0.0,0.0,2025-11-02
31536,56,81640,Warren AFB,0.0,0.0,0.0,0.0,2025-11-02
31537,56,86737,Y-O Ranch,0.0,0.0,0.0,0.0,2025-11-02
31538,56,75355,Table Rock,0.0,0.0,0.0,0.0,2025-11-02


In [4]:
city_parks.osm_area_m2.sum()

0.0